In [1]:
import json 
import numpy as np

In [2]:
with open("./data_framepassage_test-deltatime_0.07_.json") as f:
    frame_passage_object = json.load(f)

with open("./data_object_tracking_test-deltatime_0.07_.json") as f:
    object_tracking = json.load(f)

In [3]:
frame_passage_object

{'1': [],
 '2': [],
 '3': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 '4': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 '5': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 '6': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 '7': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 '8': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 '9': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18],
 '10': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 22],
 '11': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 22],
 '12': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 22],
 '13': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 22, 23],
 '14': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 22, 23],
 '15': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 22, 23],
 '16': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 18],
 '17': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 18],
 '18': [1, 

On va définir les events de la manière suivante (déf naïf) pour un objet: 
* sans passage : soit immobile
* passage : l'objet bouge constamment dans la vidéo et la norme de vitesse est au-dessus d'un seuil que l'on estimera
* en arrêt : l'objet bouge puis s'arrête 
* en démarrage : l'objet est immobile puis bouge

En raison de la symétrie des deux derniers events, il est difficile de dire lequel des events un objet correspond dans le temps 
=> peut-être prendre des blocs de frame dans l'analyse

Deux stratégies : 
* considère les frames individuellement et donne un label pour chaque objet détecter dans la frame, s'il y a event de passage, arrêt ou démarrage : les labels données en sorties sera une combinaison, ex: démarrage voiture + passage, passage plusieurs voitures, ....

* considère un bloc de frame fondamental (sorte de moyennisation) et effectue le même traitement que la stratégie précédente

In [4]:
object_tracking.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '18', '22', '23', '27', '30', '34', '39', '40', '41', '45', '51', '52', '54', '58', '61', '63', '70', '71', '73', '79', '81', '82', '84', '90', '92', '94', '99', '102', '103', '104', '105', '108', '111', '114', '117', '118', '121', '126', '127', '128'])

In [5]:
# On a clairement besoin de rajouter la vélocité des objets dans tout ça
import copy 
def velocity(json,delta_time=0.07,px_to_meter=0.1):
    json_copy = copy.deepcopy(json)
    for object in json_copy.keys():
        frames=list(json_copy[object].keys())
        for t in range(len(frames)):
        
            if t>=1:
                x_velocity = (json_copy[object][frames[t]][0] - json_copy[object][frames[t-1]][0]) * px_to_meter / ((int(frames[t]) - int(frames[t-1])) * delta_time)    
                y_velocity = (json_copy[object][frames[t]][1] - json_copy[object][frames[t-1]][1]) * px_to_meter / ((int(frames[t]) - int(frames[t-1])) * delta_time)
                
                velocity_norm=np.sqrt(x_velocity**2 + y_velocity**2)

                json_copy[object][frames[t]].extend([round(x_velocity,2),round(y_velocity,2),round(velocity_norm,2)])
            else : 
                json_copy[object][frames[t]].extend([0,0,0])
    return json_copy

def acc(json,delta_time=0.07):
    json_copy = copy.deepcopy(json)
    for object in json_copy.keys():
        frames=list(json_copy[object].keys())
        for t in range(len(frames)):
            #print(json_copy[object][frame+1])
            
            if t>=1:
                x_acc = (json_copy[object][frames[t]][5] - json_copy[object][frames[t-1]][7]) / ((int(frames[t]) - int(frames[t-1])) * delta_time)        
                y_acc = (json_copy[object][frames[t]][6] - json_copy[object][frames[t-1]][6]) / ((int(frames[t]) - int(frames[t-1])) * delta_time)    
                
                acc_norm=np.sqrt(x_acc**2 + y_acc**2)

                json_copy[object][frames[t]].extend([round(x_acc,2),round(y_acc,2),round(acc_norm,2)])
            else : 
                json_copy[object][frames[t]].extend([0,0,0])
    return json_copy 

object_tracking_treated=acc(velocity(object_tracking))

In [6]:
# 40 ne correspond à rien (l'arbre fait des choses bizarres)
object_tracking_treated["40"]

{'122': [428.5, 145.0, 29, 58, 'person', 0, 0, 0, 0, 0, 0],
 '123': [428.5, 145.0, 31, 58, 'person', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '124': [428.0, 145.0, 30, 58, 'person', -7.14, 0.0, 7.14, -102.0, 0.0, 102.0],
 '125': [428.5,
  144.5,
  31,
  59,
  'person',
  7.14,
  -7.14,
  10.1,
  0.0,
  -102.0,
  102.0],
 '126': [428.5,
  145.0,
  31,
  58,
  'person',
  0.0,
  7.14,
  7.14,
  -144.29,
  204.0,
  249.87],
 '127': [428.5, 145.5, 31, 59, 'person', 0.0, 7.14, 7.14, -102.0, 0.0, 102.0]}

In [7]:
# 18 aurait du etre une personne et pas une video 
object_tracking_treated["18"]

{'4': [177.0, 372.0, 8, 24, 'car', 0, 0, 0, 0, 0, 0],
 '5': [178.5,
  371.0,
  9,
  26,
  'car',
  21.43,
  -14.29,
  25.75,
  306.14,
  -204.14,
  367.96],
 '6': [179.5,
  369.5,
  9,
  25,
  'car',
  14.29,
  -21.43,
  25.75,
  -163.71,
  -102.0,
  192.89],
 '7': [181.5, 369.5, 9, 25, 'car', 28.57, 0.0, 28.57, 40.29, 306.14, 308.78],
 '8': [182.0,
  368.5,
  8,
  23,
  'car',
  7.14,
  -14.29,
  15.97,
  -306.14,
  -204.14,
  367.96],
 '9': [184.5, 368.5, 9, 23, 'car', 35.71, 0.0, 35.71, 282.0, 204.14, 348.14],
 '10': [185.5,
  368.0,
  9,
  24,
  'car',
  14.29,
  -7.14,
  15.97,
  -306.0,
  -102.0,
  322.55],
 '11': [186.5, 368.0, 9, 24, 'car', 14.29, 0.0, 14.29, -24.0, 102.0, 104.79],
 '12': [187.5, 367.0, 9, 24, 'car', 14.29, -14.29, 20.2, 0.0, -204.14, 204.14],
 '13': [189.0, 366.0, 10, 24, 'car', 21.43, -14.29, 25.75, 17.57, 0.0, 17.57],
 '14': [190.0,
  366.0,
  10,
  24,
  'car',
  14.29,
  0.0,
  14.29,
  -163.71,
  204.14,
  261.68],
 '15': [191.0, 365.5, 10, 25, 'car', 14.

In [8]:
# Un exemple de voiture
object_tracking_treated["1"]

{'3': [526.5, 276.5, 45, 35, 'car', 0, 0, 0, 0, 0, 0],
 '4': [526.5, 276.5, 45, 35, 'car', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '5': [526.5, 276.5, 45, 35, 'car', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '6': [526.5, 276.5, 45, 35, 'car', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '7': [526.5, 276.5, 45, 35, 'car', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '8': [526.0, 276.5, 44, 35, 'car', -7.14, 0.0, 7.14, -102.0, 0.0, 102.0],
 '9': [526.0, 276.5, 44, 35, 'car', 0.0, 0.0, 0.0, -102.0, 0.0, 102.0],
 '10': [525.0, 276.5, 44, 35, 'car', -14.29, 0.0, 14.29, -204.14, 0.0, 204.14],
 '11': [525.0, 276.5, 44, 35, 'car', 0.0, 0.0, 0.0, -204.14, 0.0, 204.14],
 '12': [525.0, 276.5, 44, 35, 'car', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 '13': [525.5, 276.5, 45, 35, 'car', 7.14, 0.0, 7.14, 102.0, 0.0, 102.0],
 '14': [525.5, 276.5, 45, 35, 'car', 0.0, 0.0, 0.0, -102.0, 0.0, 102.0],
 '15': [526.5, 276.5, 45, 35, 'car', 14.29, 0.0, 14.29, 204.14, 0.0, 204.14],
 '16': [526.5, 276.5, 45, 35, 'car', 0.0, 0.0, 0.0, -204.14, 0.0, 204.14],
 '17': 

In [9]:
#from sklearn.cluster import KMEANS
#def correction_labelisation(json):
#    object_to_label={}
#    for object in json.keys():
#        #prend première frame de chaque objet et stock dans un dict pour plus tard comparer
#        object_to_label[object]=[json[object][0][3],json[object][0][4],json[object][0][5]]
#
#    # La comparaison : on effectue un Kmeans/ pour rassembler les objets selon leur largeur et longeur de leur boîte englobattante => et on leur assigne finalement le label majoritaire
    

In [9]:
object_tracking_treated["18"]

{'4': [177.0, 372.0, 8, 24, 'car', 0, 0, 0, 0, 0, 0],
 '5': [178.5,
  371.0,
  9,
  26,
  'car',
  21.43,
  -14.29,
  25.75,
  306.14,
  -204.14,
  367.96],
 '6': [179.5,
  369.5,
  9,
  25,
  'car',
  14.29,
  -21.43,
  25.75,
  -163.71,
  -102.0,
  192.89],
 '7': [181.5, 369.5, 9, 25, 'car', 28.57, 0.0, 28.57, 40.29, 306.14, 308.78],
 '8': [182.0,
  368.5,
  8,
  23,
  'car',
  7.14,
  -14.29,
  15.97,
  -306.14,
  -204.14,
  367.96],
 '9': [184.5, 368.5, 9, 23, 'car', 35.71, 0.0, 35.71, 282.0, 204.14, 348.14],
 '10': [185.5,
  368.0,
  9,
  24,
  'car',
  14.29,
  -7.14,
  15.97,
  -306.0,
  -102.0,
  322.55],
 '11': [186.5, 368.0, 9, 24, 'car', 14.29, 0.0, 14.29, -24.0, 102.0, 104.79],
 '12': [187.5, 367.0, 9, 24, 'car', 14.29, -14.29, 20.2, 0.0, -204.14, 204.14],
 '13': [189.0, 366.0, 10, 24, 'car', 21.43, -14.29, 25.75, 17.57, 0.0, 17.57],
 '14': [190.0,
  366.0,
  10,
  24,
  'car',
  14.29,
  0.0,
  14.29,
  -163.71,
  204.14,
  261.68],
 '15': [191.0, 365.5, 10, 25, 'car', 14.

In [10]:
frame_passage_object

{'1': [],
 '2': [],
 '3': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 '4': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 '5': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 '6': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 '7': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 '8': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 '9': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18],
 '10': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 22],
 '11': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 22],
 '12': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 22],
 '13': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 22, 23],
 '14': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 22, 23],
 '15': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 22, 23],
 '16': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 18],
 '17': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 18],
 '18': [1, 

In [53]:
from collections import Counter
label_to_event = {0:"immobile",1:"mouvement",2:"arret",3:"demarrage"}
def labelisation_naif(object_tracking_json,frame_passage_object,seuil_acc_immobile=100,seuil_acc_proche=500,seuil_vitesse_immobile=20):
    
    # Voit quels objets ont été identifiés dans chaque groupe de frame
    # On skip les 4 premières frames car souvent il n'y a rien détecter dessus et calcul vitesse + acc
    result_per_frame = {}
    
    for frame_passage in list(frame_passage_object.keys())[2:]: 
        frame=frame_passage
        #print(frame)
        objects_detected = frame_passage_object[frame_passage]
        #print(objects_detected)
        frame_result_objects = []
        if objects_detected!=[]:
            #labelisation pour chaque frame
            for object in objects_detected: 
                
                object_str=str(object)
                
                frames=list(object_tracking_json[object_str].keys())
                t = frames.index(frame)

                #print(object_tracking_json[object_str][frame])
                velocity_norm_t = object_tracking_json[object_str][frame][7]

                acc_norm_t = object_tracking_json[object_str][frame][-1]
                
                acc_norm_t_1 = object_tracking_json[object_str][frames[t-1]][-1]
                
                #cas immobile ou à vitesse constante
                if (acc_norm_t_1<seuil_acc_immobile and acc_norm_t <seuil_acc_immobile): 
                    if velocity_norm_t < seuil_vitesse_immobile:
                        frame_result_objects.append(0) # "immobile"
                    else:     
                        frame_result_objects.append(1) # "en_mouvement"
                #cas accélération : positive = démarrage, négative = arrêt
                else : 
                    if np.sign(acc_norm_t-acc_norm_t_1) > 0 and (acc_norm_t_1 < seuil_acc_immobile):
                        frame_result_objects.append(3) # "demarrage"
                    elif np.sign(acc_norm_t-acc_norm_t_1) < 0 and (acc_norm_t < seuil_acc_immobile):
                        frame_result_objects.append(2) # "arret"
                    else :
                        # cas accélération qui grandit ou décroit sans a priori s'arrêter/démarrer : en mouvement
                        frame_result_objects.append(1)
        frame_result_objects.append(0)

        counter_frame_label = Counter(frame_result_objects)
        #labelisation globale par groupe
        res = f"immobile={counter_frame_label[0]}, mouvement={counter_frame_label[1]}, arret={counter_frame_label[2]}, demarrage={counter_frame_label[3]}"
        result_per_frame[frame] = res

    return result_per_frame



In [55]:

labelisation=labelisation_naif(object_tracking_treated,frame_passage_object,seuil_acc_immobile=500,seuil_acc_proche=600,seuil_vitesse_immobile=20)
labelisation

{'3': 'immobile=17, mouvement=0, arret=0, demarrage=0',
 '4': 'immobile=17, mouvement=1, arret=0, demarrage=0',
 '5': 'immobile=16, mouvement=1, arret=0, demarrage=1',
 '6': 'immobile=15, mouvement=2, arret=1, demarrage=0',
 '7': 'immobile=17, mouvement=1, arret=0, demarrage=0',
 '8': 'immobile=15, mouvement=1, arret=0, demarrage=2',
 '9': 'immobile=13, mouvement=3, arret=0, demarrage=0',
 '10': 'immobile=15, mouvement=0, arret=2, demarrage=0',
 '11': 'immobile=17, mouvement=0, arret=0, demarrage=0',
 '12': 'immobile=16, mouvement=1, arret=0, demarrage=0',
 '13': 'immobile=17, mouvement=1, arret=0, demarrage=0',
 '14': 'immobile=18, mouvement=0, arret=0, demarrage=0',
 '15': 'immobile=17, mouvement=2, arret=0, demarrage=0',
 '16': 'immobile=12, mouvement=1, arret=0, demarrage=2',
 '17': 'immobile=13, mouvement=1, arret=1, demarrage=0',
 '18': 'immobile=13, mouvement=1, arret=1, demarrage=0',
 '19': 'immobile=14, mouvement=1, arret=0, demarrage=1',
 '20': 'immobile=14, mouvement=2, arre